In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import pickle

df_dow_jones = pd.read_html("https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average")[1]
display(df_dow_jones)

dow_jones_tickers = df_dow_jones['Symbol'].tolist()
dow_jones_tickers += ['^DJI']

start_date = "2023-01-01"
# start_date = "2020-08-31"
end_date = "2024-03-31"
# end_date = datetime.today().strftime('%Y-%m-%d')
path = '../data/'
t = 1 # returns of 1-day

,Company,Exchange,Symbol,Industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,1.54%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.64%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,4.80%
3,Amazon,NASDAQ,AMZN,Retailing,2024-02-26,NaN,2.93%
4,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,3.04%
5,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,3.36%
6,Caterpillar,NYSE,CAT,Construction and mining,1991-05-06,NaN,5.45%
7,Chevron,NYSE,CVX,Petroleum industry,2008-02-19,Also 1930-07-18 to 1999-11-01,2.59%
8,Cisco,NASDAQ,CSCO,Information technology,2009-06-08,NaN,0.81%
9,Coca-Cola,NYSE,KO,Drink industry,1987-03-12,Also 1932-05-26 to 1935-11-20,1.02%


In [2]:
# Generate daily PV

df_daily = yf.download(dow_jones_tickers, start=start_date, end=end_date)

DayList = df_daily.index.tolist()
PVList = df_daily.columns.get_level_values(0).unique().tolist()
SymbolList = df_daily.columns.get_level_values(1).unique().tolist()

arr_daily = np.transpose(df_daily.to_numpy().reshape(len(DayList), len(PVList), len(SymbolList)), axes=(0, 2, 1))
print(arr_daily.shape)

[*********************100%***********************]  31 of 31 completed
(311, 31, 6)


In [3]:
np.isnan(arr_daily).sum()

# No nan value

0

In [4]:
# Generate 1-day return

TAxis = 0
def tsdelay(x, t):
    y = np.roll(x, t, axis=TAxis)
    y[(slice(None),)*TAxis + (slice(None,t),)] = np.nan
    return y
def tsreturns(x, t):
    return x /  tsdelay(x, t) - 1

r1 = tsreturns(arr_daily[:,:,PVList.index('Adj Close')], t=t)

In [5]:
vars = {
    'DayList':      DayList[:-t], 
    'SymbolList':   SymbolList, 
    'PVList':       PVList, 
}

In [6]:
# Save all data

np.save(path + '^DJIPV.npy', arr_daily[:-t])
np.save(path + '^DJIr1.npy', r1[t:])
with open(path + 'Lists.pkl', 'wb') as f:
    pickle.dump(vars, f)

pd.DataFrame(r1[t:], columns=SymbolList, index=DayList[:-t]).to_csv(path + '^DJIr1.csv')

In [7]:
# pd.DataFrame(r1[t:], columns=SymbolList).cov()